<a href="https://colab.research.google.com/github/Aditya-Deshwal23/IPL-EDA/blob/main/djscecodeai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile ipl_eda.py
import argparse
import os
import pandas as pd
import plotly.express as px
import plotly.io as pio

def run_eda(input_csv, out_dir, min_balls_bat=50, min_balls_bowl=30):
    os.makedirs(out_dir, exist_ok=True)

    print("📂 Loading data …")
    df = pd.read_csv(input_csv, low_memory=False)
    print("✅ Data loaded:", df.shape)

    # --- Handle flexible columns ---
    col_map = {
        "match_id": "match_id",
        "inning": "inning",
        "over": "over",
        "ball": "ball",
        "batting_team": "batting_team",
        "bowling_team": "bowling_team",
        "batsman": "batsman",
        "bowler": "bowler",
        "batsman_runs": "batsman_runs",
        "total_runs": "total_runs",
        "player_dismissed": "player_dismissed"
    }
    for c in list(col_map.keys()):
        if c not in df.columns:
            print(f"⚠️ Column {c} missing, filling with defaults")
            if "runs" in c:
                df[c] = 0
            else:
                df[c] = "Unknown"

    # --- Basic summary ---
    summary = df.describe(include="all")
    summary.to_csv(os.path.join(out_dir, "summary.csv"))
    print("📊 Summary saved")

    # --- Top batsmen ---
    batsmen_stats = df.groupby("batsman")["batsman_runs"].agg(["sum", "count"])
    batsmen_stats = batsmen_stats[batsmen_stats["count"] >= min_balls_bat].sort_values("sum", ascending=False)

    fig = px.bar(batsmen_stats.head(10).reset_index(),
                 x="batsman", y="sum",
                 title="Top Batsmen by Runs",
                 text="sum")
    pio.write_html(fig, file=os.path.join(out_dir, "top_batsmen.html"))

    # --- Top bowlers ---
    bowlers = df.groupby("bowler")["player_dismissed"].count().sort_values(ascending=False)
    fig2 = px.bar(bowlers.head(10).reset_index(),
                  x="bowler", y="player_dismissed",
                  title="Top Bowlers by Wickets",
                  text="player_dismissed")
    pio.write_html(fig2, file=os.path.join(out_dir, "top_bowlers.html"))

    print("✅ EDA completed. Results in", out_dir)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--input", type=str, required=True, help="Input CSV file")
    parser.add_argument("--out", type=str, required=True, help="Output folder")
    parser.add_argument("--min_balls_bat", type=int, default=50)
    parser.add_argument("--min_balls_bowl", type=int, default=30)

    args = parser.parse_args()
    run_eda(args.input, args.out, args.min_balls_bat, args.min_balls_bowl)


Writing ipl_eda.py


In [ ]:
import pandas as pd

data = [
    [1,1,1,1,"Mumbai Indians","CSK","Rohit Sharma","Deepak Chahar",0,0,None],
    [1,1,1,2,"Mumbai Indians","CSK","Rohit Sharma","Deepak Chahar",4,4,None],
    [1,1,1,3,"Mumbai Indians","CSK","Rohit Sharma","Deepak Chahar",6,6,None],
    [1,1,1,4,"Mumbai Indians","CSK","Quinton de Kock","Deepak Chahar",0,0,"Quinton de Kock"],
    [1,1,1,5,"Mumbai Indians","CSK","Quinton de Kock","Deepak Chahar",1,1,None],
]

cols = ["match_id","inning","over","ball","batting_team","bowling_team",
        "batsman","bowler","batsman_runs","total_runs","player_dismissed"]

df = pd.DataFrame(data, columns=cols)
df.to_csv("ipl_deliveries.csv", index=False)

print("✅ Created sample ipl_deliveries.csv")
print(df.head())


✅ Created sample ipl_deliveries.csv
   match_id  inning  over  ball    batting_team bowling_team          batsman  \
0         1       1     1     1  Mumbai Indians          CSK     Rohit Sharma   
1         1       1     1     2  Mumbai Indians          CSK     Rohit Sharma   
2         1       1     1     3  Mumbai Indians          CSK     Rohit Sharma   
3         1       1     1     4  Mumbai Indians          CSK  Quinton de Kock   
4         1       1     1     5  Mumbai Indians          CSK  Quinton de Kock   

          bowler  batsman_runs  total_runs player_dismissed  
0  Deepak Chahar             0           0             None  
1  Deepak Chahar             4           4             None  
2  Deepak Chahar             6           6             None  
3  Deepak Chahar             0           0  Quinton de Kock  
4  Deepak Chahar             1           1             None  


In [ ]:
pip install plotly pandas


In [ ]:
%%writefile ipl_eda.py
"""
IPL Deliveries – Advanced EDA Script (Interactive Plotly + Auto Insights)

Usage (example):
  python ipl_eda.py --input data/ipl_deliveries.csv --out out_ipl_eda

What this script does:
1) Loads a ball‑by‑ball "deliveries" dataset (robust to common column name variations).
2) Cleans & engineers features (standardized names, over/ball numbers, phases, wickets, boundaries).
3) Produces key analyses:
   - Top batters (runs, SR, average) with thresholds
   - Top bowlers (wkts, econ, dot%) with proper dismissal credit
   - Team phase performance (Powerplay/Middle/Death): run rate, wickets lost, boundary rate
   - Bowler–Batter matchups (balls, runs, dismissals, SR) and a ranked "nemesis" table
4) Saves interactive Plotly visuals and an autogenerated insights report.
5) Builds a mini static dashboard (index.html) linking all charts + insights.

Notes:
- Works with common Kaggle/IPL deliveries schemas. If your columns differ, see the COLUMN_ALIASES map below.
- Requires: pandas, numpy, plotly

Author: (you)
"""
from __future__ import annotations
import argparse
from pathlib import Path
import os
import json
import textwrap
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# --------------------------- Configurable Column Aliases ---------------------------
# Map our canonical names to possible alternatives found in IPL deliveries datasets.
COLUMN_ALIASES = {
    "match_id": ["match_id", "id", "matchId", "match_code", "match"],
    "inning": ["inning", "innings"],
    "batting_team": ["batting_team", "battingTeam", "team_batting", "BattingTeam"],
    "bowling_team": ["bowling_team", "bowlingTeam", "team_bowling", "BowlingTeam"],
    "batter": ["batter", "batsman", "striker", "Striker"],
    "non_striker": ["non_striker", "non-striker", "nonStriker", "NonStriker"],
    "bowler": ["bowler", "Bowler"],
    "over": ["over", "Over"],  # usually 0..19 or 1..20
    "ball": ["ball", "Ball", "ball_in_over", "ballnumber", "ballNumber"],
    "batsman_runs": ["batsman_runs", "batter_runs", "runs_off_bat", "Runs.Batter", "BatsmanRuns"],
    "extra_runs": ["extra_runs", "Extras", "extras", "total_extras"],
    "total_runs": ["total_runs", "TotalRuns", "runs_total", "runs"],
    "is_wicket": ["is_wicket", "isWicket", "wicket", "Wicket"],
    "player_dismissed": ["player_dismissed", "playerDismissed", "dismissed_player", "DismissedPlayer"],
    "dismissal_kind": ["dismissal_kind", "dismissalKind", "kind_of_dismissal", "DismissalKind"],
    "fielder": ["fielder", "Fielder"],
    # Optional match metadata (if present)
    "venue": ["venue", "Venue"],
    "city": ["city", "City"],
    "date": ["date", "match_date", "Date"],
    "season": ["season", "Season", "year"],
}

# Dismissals credited to bowler (ignore run-out & some others)
BOWLER_DISMISSAL_KINDS = {
    "bowled", "caught", "lbw", "stumped", "hit wicket", "caught and bowled", "hitwicket",
}

# --------------------------- Helpers ---------------------------

def find_first_present_column(df: pd.DataFrame, candidates: list[str]) -> str | None:
    for c in candidates:
        if c in df.columns:
            return c
    return None


def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    mapping = {}
    for canonical, candidates in COLUMN_ALIASES.items():
        col = find_first_present_column(df, candidates)
        if col is not None:
            mapping[col] = canonical
    sdf = df.rename(columns=mapping).copy()

    # Minimal required columns
    required = [
        "match_id", "inning", "batting_team", "bowling_team",
        "batter", "bowler", "over", "batsman_runs", "total_runs",
    ]
    missing = [c for c in required if c not in sdf.columns]
    if missing:
        raise ValueError(f"Missing required columns after standardization: {missing}\n"
                         f"Available columns: {list(sdf.columns)}\n"
                         f"Update COLUMN_ALIASES to map your dataset.")

    # Optional fallbacks
    if "is_wicket" not in sdf.columns:
        # Derive from player_dismissed, if available
        if "player_dismissed" in sdf.columns:
            sdf["is_wicket"] = sdf["player_dismissed"].notna().astype(int)
        else:
            sdf["is_wicket"] = 0

    if "extra_runs" not in sdf.columns:
        sdf["extra_runs"] = sdf["total_runs"] - sdf["batsman_runs"]

    # Ensure numeric types
    for numcol in ["inning", "over", "ball", "batsman_runs", "extra_runs", "total_runs", "is_wicket"]:
        if numcol in sdf.columns:
            sdf[numcol] = pd.to_numeric(sdf[numcol], errors="coerce")

    # Normalize team/player text
    for tcol in ["batting_team", "bowling_team", "batter", "bowler"]:
        sdf[tcol] = sdf[tcol].astype(str).str.strip()

    # Some datasets have over as 0..19; others 1..20. Store 1..20 for phase logic
    # We also try to extract ball_in_over as 1..6
    if "ball" not in sdf.columns or sdf["ball"].isna().all():
        # Sometimes ball number is encoded as decimal part of another col; skip here
        pass

    # Derive clean over number in 1..20
    over_min = sdf["over"].min()
    if over_min == 0:
        sdf["over_number"] = sdf["over"] + 1
    else:
        sdf["over_number"] = sdf["over"]

    # Best effort ball-in-over
    if "ball" in sdf.columns and sdf["ball"].notna().any():
        # Clamp to 1..10 just in case
        sdf["ball_in_over"] = sdf["ball"].fillna(1).astype(int).clip(1, 10)
    else:
        # If absent, try to sequence within each over
        sdf["ball_in_over"] = (
            sdf.sort_values(["match_id", "inning", "over_number"])\
               .groupby(["match_id", "inning", "over_number"])\
               .cumcount() + 1
        )

    # Phase bucketing
    def phase_from_over(o: float) -> str:
        try:
            o = int(o)
        except Exception:
            return "Unknown"
        if 1 <= o <= 6:
            return "Powerplay (1-6)"
        if 7 <= o <= 15:
            return "Middle (7-15)"
        if 16 <= o <= 20:
            return "Death (16-20)"
        return "Unknown"

    sdf["phase"] = sdf["over_number"].apply(phase_from_over)

    # Feature flags
    sdf["is_boundary"] = sdf["batsman_runs"].isin([4, 6]).astype(int)
    sdf["is_dot"] = (sdf["total_runs"] == 0).astype(int)

    # Bowler-credited wicket flag
    if "dismissal_kind" in sdf.columns:
        dk = sdf["dismissal_kind"].astype(str).str.lower().str.strip()
        sdf["is_bowler_wicket"] = (
            sdf["is_wicket"].eq(1) & dk.isin(BOWLER_DISMISSAL_KINDS)
        ).astype(int)
    else:
        sdf["is_bowler_wicket"] = sdf["is_wicket"]

    return sdf


def safe_div(n, d):
    return float(n) / d if d not in (0, None, np.nan) and d != 0 else 0.0

# --------------------------- Analyses ---------------------------

def top_batters(df: pd.DataFrame, min_balls: int = 200) -> pd.DataFrame:
    grp = df.groupby("batter", as_index=False).agg(
        runs=("batsman_runs", "sum"),
        balls=("batsman_runs", "count"),
        boundaries=("is_boundary", "sum"),
        dismissals=("player_dismissed", lambda s: s.eq(s.name).sum() if "player_dismissed" in df.columns else 0),
    )
    # In case lambda above fails due to scope, recompute dismissals properly
    if "player_dismissed" in df.columns:
        dism = df[df["player_dismissed"].notna()].groupby("player_dismissed").size().rename("dismissals")
        grp = grp.drop(columns="dismissals", errors="ignore").merge(dism, how="left", left_on="batter", right_index=True)
        grp["dismissals"] = grp["dismissals"].fillna(0).astype(int)
    else:
        grp["dismissals"] = 0

    grp["SR"] = (grp["runs"] * 100 / grp["balls"]).round(2)
    grp["Avg"] = grp.apply(lambda r: round(safe_div(r["runs"], r["dismissals"]) if r["dismissals"] > 0 else np.nan, 2), axis=1)
    grp["4s+6s_per_100b"] = (grp["boundaries"] * 100 / grp["balls"]).round(2)
    out = grp[grp["balls"] >= min_balls].sort_values(["runs", "SR"], ascending=[False, False])
    return out


def top_bowlers(df: pd.DataFrame, min_balls: int = 300) -> pd.DataFrame:
    grp = df.groupby("bowler", as_index=False).agg(
        balls=("total_runs", "count"),
        runs_conceded=("total_runs", "sum"),
        wkts=("is_bowler_wicket", "sum"),
        dots=("is_dot", "sum"),
    )
    grp["overs"] = (grp["balls"] / 6).round(1)
    grp["econ"] = grp.apply(lambda r: round(safe_div(r["runs_conceded"], r["balls"]) * 6, 2), axis=1)
    grp["SR"] = grp.apply(lambda r: round(safe_div(r["balls"], r["wkts"]) if r["wkts"] > 0 else np.nan, 2), axis=1)
    grp["dot%"] = (grp.apply(lambda r: safe_div(r["dots"], r["balls"]) * 100, axis=1)).round(2)
    out = grp[grp["balls"] >= min_balls].sort_values(["wkts", "econ"], ascending=[False, True])
    return out


def team_phase_metrics(df: pd.DataFrame) -> pd.DataFrame:
    grp = df.groupby(["batting_team", "phase"], as_index=False).agg(
        runs=("total_runs", "sum"),
        balls=("total_runs", "count"),
        wkts=("is_wicket", "sum"),
        boundaries=("is_boundary", "sum"),
    )
    grp["RR"] = (grp["runs"] * 6 / grp["balls"]).round(2)
    grp["wickets_per_over"] = (grp.apply(lambda r: safe_div(r["wkts"], r["balls"]) * 6, axis=1)).round(2)
    grp["boundary%"] = (grp.apply(lambda r: safe_div(r["boundaries"], r["balls"]) * 100, axis=1)).round(2)
    return grp


def bowler_death_specialists(df: pd.DataFrame, min_balls: int = 150) -> pd.DataFrame:
    death = df[df["phase"] == "Death (16-20)"]
    grp = death.groupby("bowler", as_index=False).agg(
        balls=("total_runs", "count"),
        runs_conceded=("total_runs", "sum"),
        wkts=("is_bowler_wicket", "sum"),
    )
    grp = grp[grp["balls"] >= min_balls]
    grp["econ"] = grp.apply(lambda r: round(safe_div(r["runs_conceded"], r["balls"]) * 6, 2), axis=1)
    grp["SR"] = grp.apply(lambda r: round(safe_div(r["balls"], r["wkts"]) if r["wkts"] > 0 else np.nan, 2), axis=1)
    return grp.sort_values(["econ", "SR"], ascending=[True, True])


def batter_powerplay_kings(df: pd.DataFrame, min_balls: int = 150) -> pd.DataFrame:
    pp = df[df["phase"] == "Powerplay (1-6)"]
    grp = pp.groupby("batter", as_index=False).agg(
        runs=("batsman_runs", "sum"),
        balls=("batsman_runs", "count"),
        dismissals=("player_dismissed", lambda s: s.eq(s.name).sum() if "player_dismissed" in df.columns else 0),
        boundaries=("is_boundary", "sum"),
    )
    # Recompute dismissals via player_dismissed index
    if "player_dismissed" in df.columns:
        dism = pp[pp["player_dismissed"].notna()].groupby("player_dismissed").size().rename("dismissals")
        grp = grp.drop(columns="dismissals", errors="ignore").merge(dism, how="left", left_on="batter", right_index=True)
        grp["dismissals"] = grp["dismissals"].fillna(0).astype(int)
    else:
        grp["dismissals"] = 0

    grp = grp[grp["balls"] >= min_balls]
    grp["SR"] = (grp["runs"] * 100 / grp["balls"]).round(2)
    grp["boundary%"] = (grp["boundaries"] * 100 / grp["balls"]).round(2)
    grp["Avg"] = grp.apply(lambda r: round(safe_div(r["runs"], r["dismissals"]) if r["dismissals"] > 0 else np.nan, 2), axis=1)
    return grp.sort_values(["SR", "runs"], ascending=[False, False])


def matchup_table(df: pd.DataFrame, min_balls: int = 30) -> pd.DataFrame:
    grp = df.groupby(["batter", "bowler"], as_index=False).agg(
        runs=("batsman_runs", "sum"),
        balls=("batsman_runs", "count"),
        dismissals=("is_bowler_wicket", "sum"),
        boundaries=("is_boundary", "sum"),
    )
    grp = grp[grp["balls"] >= min_balls]
    grp["SR"] = (grp["runs"] * 100 / grp["balls"]).round(2)
    grp["dismissal_rate%"] = (grp["dismissals"] * 100 / grp["balls"]).round(2)
    # Rank harsh matchups for batters: high dismissal rate & low SR
    grp["nemesis_score"] = (grp["dismissal_rate%"] - (grp["SR"] / 2)).round(2)
    return grp.sort_values(["nemesis_score", "dismissal_rate%"], ascending=[False, False])

# --------------------------- Visualizations ---------------------------

def save_plot(fig: go.Figure, out_dir: Path, name: str) -> str:
    out_path = out_dir / f"{name}.html"
    fig.write_html(str(out_path), include_plotlyjs="cdn")
    return str(out_path)


def viz_top_batters(df_bat: pd.DataFrame, out_dir: Path, k: int = 15) -> str:
    topk = df_bat.head(k)
    fig = px.bar(topk, x="batter", y="runs", hover_data=["SR", "Avg", "balls", "4s+6s_per_100b"],
                 title=f"Top {k} Batters by Runs (min balls threshold)")
    fig.update_layout(xaxis_title="Batter", yaxis_title="Runs", xaxis_tickangle=45)
    return save_plot(fig, out_dir, "top_batters")


def viz_top_bowlers(df_bowl: pd.DataFrame, out_dir: Path, k: int = 15) -> str:
    topk = df_bowl.head(k)
    fig = px.bar(topk, x="bowler", y="wkts", hover_data=["econ", "SR", "dot%", "balls", "overs"],
                 title=f"Top {k} Bowlers by Wickets (min balls threshold)")
    fig.update_layout(xaxis_title="Bowler", yaxis_title="Wickets", xaxis_tickangle=45)
    return save_plot(fig, out_dir, "top_bowlers")


def viz_team_phase(team_phase: pd.DataFrame, out_dir: Path) -> tuple[str, str, str]:
    fig_rr = px.bar(team_phase, x="batting_team", y="RR", color="phase", barmode="group",
                    title="Team Run Rate by Phase")
    fig_rr.update_layout(xaxis_title="Team", yaxis_title="Run Rate", xaxis_tickangle=45)

    fig_wk = px.bar(team_phase, x="batting_team", y="wickets_per_over", color="phase", barmode="group",
                    title="Team Wickets Lost per Over by Phase")
    fig_wk.update_layout(xaxis_title="Team", yaxis_title="Wickets/Over", xaxis_tickangle=45)

    fig_bd = px.bar(team_phase, x="batting_team", y="boundary%", color="phase", barmode="group",
                    title="Team Boundary% by Phase")
    fig_bd.update_layout(xaxis_title="Team", yaxis_title="Boundary %", xaxis_tickangle=45)

    return (
        save_plot(fig_rr, out_dir, "team_phase_rr"),
        save_plot(fig_wk, out_dir, "team_phase_wickets"),
        save_plot(fig_bd, out_dir, "team_phase_boundary"),
    )


def viz_matchups(df_mu: pd.DataFrame, out_dir: Path, k: int = 30) -> str:
    top = df_mu.head(k)
    fig = px.scatter(top, x="SR", y="dismissal_rate%", size="balls", color="runs",
                     hover_data=["batter", "bowler", "runs", "balls", "dismissals", "nemesis_score"],
                     title=f"Matchups – Potential Nemeses (Top {k} by score)")
    fig.update_layout(xaxis_title="Strike Rate", yaxis_title="Dismissal Rate %")
    return save_plot(fig, out_dir, "matchups_nemesis")


# --------------------------- Auto Insights ---------------------------

def auto_insights(df_bat: pd.DataFrame, df_bowl: pd.DataFrame, team_phase: pd.DataFrame,
                  pp_bat: pd.DataFrame, death_bowl: pd.DataFrame, df_mu: pd.DataFrame,
                  top_n: int = 5) -> list[str]:
    insights = []

    # Insight 1: Best PP teams by RR vs wickets lost
    pp = team_phase[team_phase["phase"] == "Powerplay (1-6)"]
    if not pp.empty:
        pp_sorted = pp.sort_values(["RR", "wickets_per_over"], ascending=[False, True]).head(5)
        tnames = ", ".join(pp_sorted["batting_team"].tolist())
        insights.append(
            f"Powerplay dominance: Top PP run rates with low wickets/over – {tnames}. Consider aggressive PP field settings and bowling changes vs these teams."
        )

    # Insight 2: Death over specialists
    if not death_bowl.empty:
        dtop = death_bowl.head(top_n)
        names = ", ".join(dtop["bowler"].tolist())
        insights.append(
            f"Death-overs specialists: {names} show elite economy/SR in overs 16–20. Use them for 17–20 or hold at least 2 overs for these bowlers."
        )

    # Insight 3: PP aggressors (batters)
    if not pp_bat.empty:
        btop = pp_bat.sort_values(["SR", "runs"], ascending=[False, False]).head(top_n)
        names = ", ".join(btop["batter"].tolist())
        insights.append(
            f"Early-overs enforcers: {names} have top PP strike rates. Opponents: start with swing/seam or hard length; keep third-man/fine-leg plans ready."
        )

    # Insight 4: Nemesis pairs
    if not df_mu.empty:
        row = df_mu.iloc[0]
        insights.append(
            f"Matchup to exploit: {row['bowler']} vs {row['batter']} – {row['dismissals']} dismissals in {row['balls']} balls with SR {row['SR']}. Use early if batter is set."
        )

    # Insight 5: Teams leaking at death
    death = team_phase[team_phase["phase"] == "Death (16-20)"]
    if not death.empty:
        leaky = death.sort_values("RR", ascending=False).head(3)
        tnames = ", ".join(leaky["batting_team"].tolist())
        insights.append(
            f"Death-over run fests: Highest RR at death against – {tnames}. Bowling teams should plan yorker/variation practice and boundary riders."
        )

    return insights[:5]


# --------------------------- Orchestration ---------------------------

def run_eda(input_csv: Path, out_dir: Path, min_balls_bat: int = 200, min_balls_bowl: int = 300,
            min_balls_mu: int = 30) -> None:
    out_dir.mkdir(parents=True, exist_ok=True)

    print("\nLoading data …")
    raw = pd.read_csv(input_csv, low_memory=False)
    print(f"Loaded {len(raw):,} rows, {len(raw.columns)} columns")

    print("Standardizing & cleaning …")
    df = standardize_columns(raw)

    # Cache cleaned data
    cleaned_path = out_dir / "cleaned_deliveries.parquet"
    try:
        df.to_parquet(cleaned_path, index=False)
    except Exception:
        # Fallback to CSV
        cleaned_path = out_dir / "cleaned_deliveries.csv"
        df.to_csv(cleaned_path, index=False)
    print(f"Saved cleaned data to {cleaned_path}")

    print("Computing analyses …")
    df_bat = top_batters(df, min_balls=min_balls_bat)
    df_bowl = top_bowlers(df, min_balls=min_balls_bowl)
    team_phase = team_phase_metrics(df)
    pp_bat = batter_powerplay_kings(df, min_balls=150)
    death_bowl = bowler_death_specialists(df, min_balls=150)
    df_mu = matchup_table(df, min_balls=min_balls_mu)

    # Save tables
    tables = {
        "top_batters.csv": df_bat,
        "top_bowlers.csv": df_bowl,
        "team_phase_metrics.csv": team_phase,
        "powerplay_batters.csv": pp_bat,
        "death_bowlers.csv": death_bowl,
        "matchups.csv": df_mu,
    }
    for name, tdf in tables.items():
        tdf.to_csv(out_dir / name, index=False)

    print("Building visuals …")
    p1 = viz_top_batters(df_bat, out_dir)
    p2 = viz_top_bowlers(df_bowl, out_dir)
    p3, p4, p5 = viz_team_phase(team_phase, out_dir)
    p6 = viz_matchups(df_mu, out_dir)

    print("Drafting insights …")
    insights = auto_insights(df_bat, df_bowl, team_phase, pp_bat, death_bowl, df_mu)
    insights_md = "\n".join([f"- {s}" for s in insights])
    (out_dir / "insights.md").write_text("# IPL EDA – Key Insights\n\n" + insights_md + "\n", encoding="utf-8")

    print("Assembling dashboard …")
    index_html = f"""
    <!doctype html>
    <html>
    <head>
      <meta charset='utf-8' />
      <meta name='viewport' content='width=device-width, initial-scale=1' />
      <title>IPL Deliveries – EDA Dashboard</title>
      <style>
        body{{font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu; margin:24px;}}
        .grid{{display:grid; gap:24px; grid-template-columns:repeat(auto-fit,minmax(320px,1fr));}}
        .card{{border:1px solid #e5e7eb; border-radius:16px; padding:16px; box-shadow:0 1px 3px rgba(0,0,0,0.06)}}
        h1{{margin-top:0}}
        a{{text-decoration:none;}}
        ul{{line-height:1.6}}
      </style>
    </head>
    <body>
      <h1>IPL Deliveries – EDA Dashboard</h1>
      <p>Auto-generated interactive visuals & insights. Click a chart to explore (hover, zoom, filter legend).</p>
      <div class='grid'>
        <div class='card'><h3>Top Batters</h3><a href='./top_batters.html' target='_blank'>Open chart</a></div>
        <div class='card'><h3>Top Bowlers</h3><a href='./top_bowlers.html' target='_blank'>Open chart</a></div>
        <div class='card'><h3>Team RR by Phase</h3><a href='./team_phase_rr.html' target='_blank'>Open chart</a></div>
        <div class='card'><h3>Team Wickets/Over</h3><a href='./team_phase_wickets.html' target='_blank'>Open chart</a></div>
        <div class='card'><h3>Team Boundary%</h3><a href='./team_phase_boundary.html' target='_blank'>Open chart</a></div>
        <div class='card'><h3>Matchups – Nemesis</h3><a href='./matchups_nemesis.html' target='_blank'>Open chart</a></div>
      </div>
      <div class='card' style='margin-top:24px;'>
        <h2>Key Insights</h2>
        <div id='insights'></div>
        <script>
          fetch('./insights.md').then(r => r.text()).then(t => {{
            const el = document.getElementById('insights');
            el.innerHTML = t.split('\n').map(line => line.startsWith('- ')? `<li>${{line.slice(2)}}</li>` : '').join('');
            el.outerHTML = `<ul>${{el.innerHTML}}</ul>`;
          }});
        </script>
      </div>
    </body>
    </html>
    """
    (out_dir / "index.html").write_text(index_html, encoding="utf-8")

    print("\nDone! Outputs saved to:", out_dir)
    print("Open the dashboard:", (out_dir / "index.html").resolve())


# --------------------------- CLI ---------------------------

def parse_args():
    p = argparse.ArgumentParser(description="IPL Deliveries – Advanced EDA (interactive)")
    p.add_argument("--input", required=True, type=Path, help="Path to deliveries CSV")
    p.add_argument("--out", required=True, type=Path, help="Output directory for charts & tables")
    p.add_argument("--min_balls_bat", type=int, default=200, help="Min balls for batter ranking")
    p.add_argument("--min_balls_bowl", type=int, default=300, help="Min balls for bowler ranking")
    p.add_argument("--min_balls_mu", type=int, default=30, help="Min balls in a matchup")
    return p.parse_args()


if __name__ == "__main__":
    args = parse_args()
    run_eda(args.input, args.out, args.min_balls_bat, args.min_balls_bowl, args.min_balls_mu)


Overwriting ipl_eda.py


In [ ]:
import pandas as pd

# Load your CSV
df = pd.read_csv("ipl_deliveries.csv")

# See how many rows you have
print("Rows:", df.shape)

# See first 5 rows
print(df.head())

# Check unique batsmen
print("Unique batsmen:", df['batsman'].unique())


In [ ]:
!python ipl_eda.py --input ipl_deliveries.csv --out out_ipl_eda



Loading data …
Loaded 5 rows, 11 columns
Standardizing & cleaning …
Saved cleaned data to out_ipl_eda/cleaned_deliveries.parquet
Computing analyses …
Building visuals …
Drafting insights …
Assembling dashboard …

Done! Outputs saved to: out_ipl_eda
Open the dashboard: /content/out_ipl_eda/index.html


In [ ]:
!ls out_ipl_eda

cleaned_deliveries.parquet  matchups_nemesis.html     team_phase_wickets.html
death_bowlers.csv	    powerplay_batters.csv     top_batters.csv
index.html		    team_phase_boundary.html  top_batters.html
insights.md		    team_phase_metrics.csv    top_bowlers.csv
matchups.csv		    team_phase_rr.html	      top_bowlers.html


In [ ]:
import plotly.express as px
import pandas as pd

df = pd.read_csv("ipl_deliveries.csv")

top_batsmen = (
    df.groupby("batsman")["batsman_runs"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .reset_index()
)

fig = px.bar(
    top_batsmen,
    x="batsman",
    y="batsman_runs",
    title="Top 10 Batsmen by Runs",
    text="batsman_runs"
)
fig.show()
